# A cautionary tale of ground state energy calculation

When calculating the ground state energy of an electronic structure Hamiltonian, some care should be exercised. Converting the Hamiltonian to a sparse matrix and finding the least eigenvalue will not always be the _true_ ground state with respect to the system it represents, since the diagonalization scheme used may not inherently observe any symmetries present in the underlying physical system.

Consider the following $H_3^+$ example, consisting of 2 electrons in 6 spin-orbitals...

In [2]:
from symmer.chemistry import MoleculeBuilder
from symmer.symplectic import array_to_QuantumState
from symmer.projection import QubitTapering

basis='sto-3g'
charge=+1
geometry =[
    ("H", [0., 0.558243000, 0.]),
    ("H", [0.483452000, -0.279121000, 0.]),
    ("H", [-0.483452000, -0.279121000, 0.])
]
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

Molecule geometry:
H	0.0	0.558243	0.0
H	0.483452	-0.279121	0.0
H	-0.483452	-0.279121	0.0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -1.246860006338445
MP2 energy:  -1.2658602663569554
CCSD energy: -1.274144616958313
FCI energy:  -1.2741444671239879


Number of qubits: 6


Naively computing the ground state energy by taking the smallest eigenvalue of the Hamiltonian does not match the FCI energy:

In [33]:
from symmer.chemistry import MoleculeBuilder
from symmer.symplectic import array_to_QuantumState

basis='sto-3g'
charge=0
geometry =[
    ("H", [0., 0., 1.]),
    ("H", [0., 1., 0]),
    ("H", [0., 1., 1.]),
    ("H", [0., 0., 0.])
]

# geometry =[
#     ("H", [0., 0., 0.]),
#     ("H", [0., 0., 0.74]),
# ]
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

Molecule geometry:
H	0.0	0.0	1.0
H	0.0	1.0	0
H	0.0	1.0	1.0
H	0.0	0.0	0.0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -1.7610750541348996
MP2 energy:  -1.81984049526728
CCSD energy: -1.9186787645494061
FCI energy:  -1.915106549511806


Number of qubits: 8


In [34]:
from symmer.utils import exact_gs_energy

gs_nrg, gs_vec = exact_gs_energy(molecule.H_q.to_sparse_matrix)

print(f'Least eigenvalue = {gs_nrg} | FCI error = {gs_nrg - molecule.fci_energy}')

Least eigenvalue = -1.9151065495117348 | FCI error = 7.127631818093505e-14


What has gone wrong here? Taking a look at the corresponding eigenvector, we see the identified state actually contains 3 particles, whereas the underlying system only contains 2:

In [35]:
psi = array_to_QuantumState(gs_vec).cleanup(zero_threshold=1e-5)
print(f'Eigenvector with eigenvalue {gs_nrg}:\n')
print(psi)

Eigenvector with eigenvalue -1.9151065495117348:

-0.044-0.007j |00001111> +
 0.044+0.007j |00110011> +
-0.060-0.010j |01011010> +
-0.060-0.010j |01100110> +
 0.120+0.020j |01101001> +
 0.120+0.020j |10010110> +
-0.060-0.010j |10011001> +
-0.060-0.010j |10100101> +
 0.681+0.111j |11001100> +
-0.681-0.111j |11110000>


To counter this issue, we instead need to select the least eigenvalue that contains the _correct_ number of particles. This is implemented in `symmer.chem.exact_gs_state`

4

In [41]:
from symmer.chemistry import exact_gs_energy

gs_nrg, gs_vec = exact_gs_energy(molecule.H_q.to_sparse_matrix, n_particles=molecule.pyscf_obj.pyscf_hf.mol.nelectron)
psi = array_to_QuantumState(gs_vec).cleanup(zero_threshold=1e-5)

print(f'Least eigenvalue = {gs_nrg} | FCI error = {gs_nrg - molecule.fci_energy}\n')
print(f'Eigenvector with eigenvalue {gs_nrg}:\n')
print(psi)

Least eigenvalue = -1.9151065495117299 | FCI error = 7.616129948928574e-14

Eigenvector with eigenvalue -1.9151065495117299:

 0.043-0.011j |00001111> +
-0.043+0.011j |00110011> +
 0.059-0.015j |01011010> +
 0.059-0.015j |01100110> +
-0.117+0.031j |01101001> +
-0.117+0.031j |10010110> +
 0.059-0.015j |10011001> +
 0.059-0.015j |10100101> +
-0.668+0.174j |11001100> +
 0.668-0.174j |11110000>


Success! We have now recovered the true ground state by enforcing that only solutions with the correct number of particles are considered. Note however it is possible that no solution is identified at first - in this case, increase the `n_eigs` parameter in `symmer.chem.exact_gs_state` to increase the search space.

In [42]:
molecule.H_q

-0.243+0.000j IIIIIIII +
 0.238+0.000j ZIIIIIII +
-0.000+0.000j YZZZZZYI +
-0.000+0.000j XZZZZZXI +
 0.238+0.000j IZIIIIII +
-0.000+0.000j IYZZZZZY +
-0.000+0.000j IXZZZZZX +
-0.009+0.000j IIZIIIII +
-0.009+0.000j IIIZIIII +
-0.009+0.000j IIIIZIII +
-0.009+0.000j IIIIIZII +
-0.313+0.000j IIIIIIZI +
-0.313+0.000j IIIIIIIZ +
 0.133+0.000j ZZIIIIII +
 0.034+0.000j YXXYIIII +
-0.034+0.000j YYXXIIII +
-0.034+0.000j XXYYIIII +
 0.034+0.000j XYYXIIII +
 0.034+0.000j YXIIXYII +
-0.034+0.000j YYIIXXII +
-0.034+0.000j XXIIYYII +
 0.034+0.000j XYIIYXII +
 0.019+0.000j YXIIIIXY +
-0.019+0.000j YYIIIIXX +
-0.019+0.000j XXIIIIYY +
 0.019+0.000j XYIIIIYX +
 0.095+0.000j ZIZIIIII +
 0.014+0.000j YZXIXZYI +
 0.014+0.000j YZYIYZYI +
 0.014+0.000j XZXIXZXI +
 0.014+0.000j XZYIYZXI +
 0.129+0.000j ZIIZIIII +
-0.000+0.000j YZZIZZYI +
-0.000+0.000j XZZIZZXI +
-0.019+0.000j YZZXIXYI +
-0.019+0.000j YZZYIYYI +
-0.019+0.000j XZZXIXXI +
-0.019+0.000j XZZYIYXI +
 0.000+0.000j YZYYZZZY +
 0.000+0.000j YZYXZZZX +


In [43]:
taper_hamiltonian = QubitTapering(molecule.H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
# print('\nand may be rotated onto the single-qubit Pauli operators\n')
# print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
# print('\nvia a sequence of rotations e^{i pi/4 R} where\n')
# for index, (rot, angle) in enumerate(taper_hamiltonian.stabilizers.stabilizer_rotations):
#     print(f'R_{index} = {rot}')

We are able to taper 3 qubits from the Hamiltonian.

The symmetry generators are

 1 ZIIZIZZI 
 1 IZIZIZIZ 
 1 IIZZZZII


In [45]:
molecule.hf_array

array([1, 1, 1, 1, 0, 0, 0, 0])

In [11]:
from openfermion import FermionOperator
import numpy as np

def parity_operators(N_qubits):
    """
    note order is assumed to be spin up, spin down, spin up, spin down ... etc
    
    each op is built as product of parities of individual sites!
    
    https://arxiv.org/pdf/1008.4346.pdf
    """
    
    parity_up = FermionOperator('', 1)
    for spin_up_ind in np.arange(0,N_qubits, 2):
        parity_up*= FermionOperator('', 1) - 2*FermionOperator(f'{spin_up_ind}^ {spin_up_ind}', 1)#
    
    parity_down = FermionOperator('',1)
    for spin_down_ind in  np.arange(1,N_qubits, 2):
        parity_down*= FermionOperator('', 1) - 2*FermionOperator(f'{spin_down_ind}^ {spin_down_ind}', 1)
    
    return parity_up, parity_down

In [18]:
from openfermion import jordan_wigner, bravyi_kitaev

up_parity, down_parity = parity_operators(len(molecule.hf_array))

In [20]:
print(jordan_wigner(up_parity))
print()
print(jordan_wigner(down_parity))

(1+0j) [Z0 Z2]

(1+0j) [Z1 Z3]


In [ ]:
# ZIZI (up)
# IZIZ (down)
# ###


# 1 ZIIZ (a)
# 1 IZIZ (b)
# 1 IIZZ (c)


# down == b

# up == (a).(c)


In [19]:
print(bravyi_kitaev(up_parity))
print()
print(bravyi_kitaev(down_parity))

(1+0j) [Z0 Z2]

(1+0j) [Z0 Z2 Z3]


In [44]:
taper_hamiltonian.stabilizers.update_sector(molecule.hf_array)

print(taper_hamiltonian.symmetry_generators)

 1 ZIIZIZZI 
 1 IZIZIZIZ 
 1 IIZZZZII


In [279]:
# molecule.H

In [21]:
molecule.hf_array

array([1, 1, 0, 0])

In [22]:
HF_state = np.eye(2**molecule.n_qubits)[int(''.join(map(str, molecule.hf_array.tolist())), 2), :]

In [23]:
from functools import reduce
test = [np.array([[0],[1]]) if bit==1 else np.array([[1],[0]]) for bit in molecule.hf_array]
test_state = reduce(np.kron, test)

np.allclose(test_state.reshape([-1]), HF_state.reshape([-1]))

True

In [24]:
from openfermion import get_sparse_operator

HF_state.conj().T @get_sparse_operator(up_parity, n_qubits=molecule.n_qubits) @ HF_state

(-1+0j)

In [25]:
HF_state.conj().T @get_sparse_operator(down_parity, n_qubits=molecule.n_qubits) @ HF_state

(-1+0j)

https://arxiv.org/pdf/1007.0586.pdf

parity operator for site $i$ is:

$$(-1)^{n_{i}}$$

aka -1 to the power of number of particles on site $i$

https://arxiv.org/pdf/1008.4346.pdf

In [137]:
n_0 =  FermionOperator(f'{0}^ {0}', 1)
n_1 =  FermionOperator(f'{1}^ {1}', 1)
n_2 =  FermionOperator(f'{2}^ {2}', 1)
n_3 =  FermionOperator(f'{3}^ {3}', 1)
ni_list = [n_0, n_1, n_2, n_3]

In [138]:
from openfermion import jordan_wigner, bravyi_kitaev

j0,j1,j2,j3 = [jordan_wigner(n) for n in ni_list]

q_list = [j0,j1,j2,j3]

In [139]:
j0

(0.5+0j) [] +
(-0.5+0j) [Z0]

In [140]:
from scipy.linalg import expm

ttt = [1j*np.pi*get_sparse_operator(op, n_qubits=4).todense() for op in q_list]

pol = reduce(lambda x,y: x+y, ttt)

exp_pol = expm(pol)
np.where(expm(exp_pol)!=0)

np.diag(exp_pol)

array([ 1.+0.j, -1.+0.j, -1.+0.j,  1.-0.j, -1.+0.j,  1.-0.j,  1.-0.j,
       -1.+0.j, -1.+0.j,  1.-0.j,  1.-0.j, -1.+0.j,  1.-0.j, -1.+0.j,
       -1.+0.j,  1.-0.j])

In [142]:
H_mat = get_sparse_operator(molecule.H).todense()

In [144]:
H_mat@exp_pol == exp_pol@H_mat

matrix([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          T

In [145]:
FermionOperator()

0

In [297]:
import os
import yaml

In [304]:
molecule.hf_array.tolist()


[1, 1, 0, 0]

In [364]:
data = {}
H_dict = molecule.H_q.to_dictionary

for key in H_dict.keys():
    H_dict[key] = float(H_dict[key].real)

# H_dict = {'cat': H_dict['IIII']}
data['H'] = H_dict
data['taper_reference'] = molecule.hf_array.tolist()

In [365]:
data

{'H': {'IIII': -0.09706626816763056,
  'IIIZ': -0.2234315369081347,
  'IIZI': -0.2234315369081347,
  'IIZZ': 0.1744128761226159,
  'IZII': 0.1714128264477691,
  'IZIZ': 0.12062523483390414,
  'IZZI': 0.1659278503377034,
  'ZIII': 0.17141282644776906,
  'ZIIZ': 0.1659278503377034,
  'ZIZI': 0.12062523483390414,
  'ZZII': 0.16868898170361205,
  'XXYY': -0.045302615503799264,
  'XYYX': 0.045302615503799264,
  'YXXY': 0.045302615503799264,
  'YYXX': -0.045302615503799264},
 'taper_reference': [1, 1, 0, 0]}

In [366]:
out = os.path.join(os.getcwd(), 'H2_JW.yaml')
with open(out, 'w') as file:
    yaml.dump(data, file)

In [367]:
with open(out, "r") as stream:
    try:
        test = yaml.full_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [368]:
test

{'H': {'IIII': -0.09706626816763056,
  'IIIZ': -0.2234315369081347,
  'IIZI': -0.2234315369081347,
  'IIZZ': 0.1744128761226159,
  'IZII': 0.1714128264477691,
  'IZIZ': 0.12062523483390414,
  'IZZI': 0.1659278503377034,
  'XXYY': -0.045302615503799264,
  'XYYX': 0.045302615503799264,
  'YXXY': 0.045302615503799264,
  'YYXX': -0.045302615503799264,
  'ZIII': 0.17141282644776906,
  'ZIIZ': 0.1659278503377034,
  'ZIZI': 0.12062523483390414,
  'ZZII': 0.16868898170361205},
 'taper_reference': [1, 1, 0, 0]}